In [8]:
import pandas as pd 

In [9]:
test_data = pd.read_csv('1061945344.csv')

In [10]:
test_data=test_data[['comment_id','article_id','content','full_name','userid','title','url']]

In [11]:
test_data

,comment_id,article_id,content,full_name,userid,title,url
0,48076303,4535124,Thức thời - lo cho mình trước đã xã hội nhà nư...,Thái Phạm,1061945344,'Nghĩ cách kiếm tiền thay vì ngồi chờ giá nhà ...,https://vnexpress.net/bai-viet-4535124.html
1,48053674,4534597,Ronaldo - CR7 là một cầu thủ tuyệt vời hoàn hả...,Thái Phạm,1061945344,HLV Bồ Đào Nha bênh vực Ronaldo - VnExpress Th...,https://vnexpress.net/hlv-bo-dao-nha-benh-vuc-...
2,48053599,4534597,Khổ quá đợi bóng lăn đã,Thái Phạm,1061945344,HLV Bồ Đào Nha bênh vực Ronaldo - VnExpress Th...,https://vnexpress.net/hlv-bo-dao-nha-benh-vuc-...
3,48033268,4534147,"Camavinga và tchoymendy mà yếu à, chỉ là thiếu...",Thái Phạm,1061945344,Pháp mang 25 cầu thủ đi dự World Cup 2022 - Vn...,https://vnexpress.net/phap-mang-25-cau-thu-di-...
4,47756969,4496272,"Có khí phách, khi xưa mình bé cha mẹ nuôi mình...",Thái Phạm,1061945344,Viễn cảnh 'bốn thanh niên chăm bốn người già' ...,https://vnexpress.net/vien-canh-bon-thanh-nien...
5,47700442,4523815,Bạn chỉ được cái nói đúng,Thái Phạm,1061945344,Có nhẫn tâm khi tôi dứt khoát chia tay bạn gái,https://vnexpress.net/bai-viet-4523815.html
6,47699274,4520285,Sướng quá nha chú,Thái Phạm,1061945344,Vợ và con trai phủ nhận mọi đóng góp của tôi c...,https://vnexpress.net/bai-viet-4520285.html
7,47692487,4524180,Chúng ta cùng chung chí hướng,Thái Phạm,1061945344,Người Việt chi nghìn tỷ đồng mua iPhone 14 ngà...,https://vnexpress.net/nguoi-viet-chi-nghin-ty-...
8,47692317,4524180,Bạn nói kiểu gì chứ tôi xài Iphone6 vẫn chạy ầ...,Thái Phạm,1061945344,Người Việt chi nghìn tỷ đồng mua iPhone 14 ngà...,https://vnexpress.net/nguoi-viet-chi-nghin-ty-...
9,47692655,4524180,Công nhân với dân văn phòng cũng cùng là làm t...,Thái Phạm,1061945344,Người Việt chi nghìn tỷ đồng mua iPhone 14 ngà...,https://vnexpress.net/nguoi-viet-chi-nghin-ty-...


In [33]:
# Combine comments with one user
def combineComments(userid,data):
    comments = test_data.loc[test_data['userid']==userid,'content'].to_list()
    comment_id	= test_data.loc[test_data['userid']==userid,'comment_id'].to_list()
    article_id	= test_data.loc[test_data['userid']==userid,'article_id'].to_list()
    title	= test_data.loc[test_data['userid']==userid,'title'].to_list()
    url = test_data.loc[test_data['userid']==userid,'url'].to_list()
    append	= [userid,comments,comment_id,article_id,title,url]
    data = data.append(pd.Series(append, index = ["userid","comments","comments_id","article_id","title",'url']), 
           ignore_index=True)
    return(data)

In [38]:
list(test_data['userid'].unique())

[1061945344]

In [39]:
userid_list = list(test_data['userid'].unique())
data = pd.DataFrame(columns = ["userid","comments","comments_id","article_id","title",'url'])

for i in userid_list:
    data=combineComments(i,data)

C:\Users\marti\AppData\Local\Temp\ipykernel_16204\218728412.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.Series(append, index = ["userid","comments","comments_id","article_id","title",'url']),


In [54]:
data['comments'].to_list()[0][1]

'Ronaldo - CR7 là một cầu thủ tuyệt vời hoàn hảo, tôi công nhận điều đó nhưng với phong độ + thái độ + trình độ của anh ở thời điểm này thì anh không xứng đáng có mặt tại WC năm nay - ý kiến riêng của tôi'

# Analyze Users' Comments

In [6]:
#call library to preprocess tweets
import nltk
import string
import re
#nltk.download()
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#ps = nltk.PorterStemmer()
#wn = nltk.WordNetLemmatizer()
stop = set(line.strip() for line in open('stop_words.txt', encoding='utf-8'))
exclude = set(string.punctuation) 

words = set(nltk.corpus.words.words())

Tiền xử lý:
  - Làm sạch loại bỏ ký tự đặc biệt và dấu câu
  - chuẩn hóa về chữ thường
  - Loại bỏ stop words
  - Thay thế các url trong dữ liệu bởi nhãn link_spam
  - Xử lý các trường hợp người dùng láy láy âm tiết(Ví dụ: Ngooon quááááá điiiiiiii !!!!!)
  - Chuẩn hóa các từ viết tắt cơ bản(Ví dụ: k, ko, k0 --> không, bt --> bình thường,...) (teencode)
  - Loại bỏ số và các từ chỉ có 1 ký tự

In [83]:
# hàm này đúng code nhưng sai logic tiền xử lý
def processComments(comments):
    
    #cleaning of comments
    cleanedcomments = []
    for comment in comments:
        cm = re.sub('http\S+', '', comment) #remove links
        
        cm = "".join([char for char in cm if char not in string.punctuation]) #remove punctuations
        
        cm = cm.lower() #converting to lowercase letters
        
        cm = ' '.join([word for word in cm.split() if word not in (stop)]) #removing stop words (có vấn đề)
        
        cm = ' '.join([word for word in cm.split() if len(word)>2])
        
        cleanedcomments.append(cm)

        

    cleanedcomments = ' '.join(cleanedcomments) #joining all comments
    

    #tokenization
    Processedcomments = nltk.word_tokenize(cleanedcomments) ####################################################

    #Processedcomments = [word for word in Processedcomments if len(word)>2]
    

    Processedcomments = ' '.join(w for w in Processedcomments)

    return Processedcomments
    
# process all tweet from all user (ptweets). then, appending into dataframe (only one dataframe)
data["pcomment"] = data['comments'].apply(lambda x : processComments(x))

thức thời hội ronaldo cr7 cầu thủ tuyệt vời hoàn hảo công phong thái trình thời xứng mặt kiến khổ đợi bóng lăn camavinga tchoymendy yếu kinh nghiệm thi đấu quốc khí phách xưa nuôi comment sướng nha chí hướng kiểu xài iphone6 chạy grab bao giao công nhân dân văn phòng thuê
